# Step 3: Cap Haitien GTFS Accessibility Analysis
How many Economic Indicators can be reached within x amount of time

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [3]:
import os, sys

In [4]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

### Load graphs

In [5]:
# read back your graphs from step 2 from you saved pickle
G_service0001 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_v5_service0001.pickle")
G_service0002 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_v5_service0002.pickle")

In [6]:
#print(nx.info(G))

In [7]:
#list(G.edges)[0:15]

## Identify only the largest graph

In [8]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G_service0001.subgraph(c).copy() for c in nx.weakly_connected_components(G_service0001))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G_service0001 = max_graph

In [9]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G_service0002.subgraph(c).copy() for c in nx.weakly_connected_components(G_service0002))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G_service0002 = max_graph

In [10]:
graphs = {'G_service0001': G_service0001, 'G_service0002': G_service0002}

In [11]:
# print info about the largest sub-graph
#print(nx.info(G_largest))

### Load Data

In [12]:
# load origins
origins = gpd.read_file(r"input_folder\cap_haitien_worldpop_pts2.shp")

In [13]:
# load destinations
destinations = gpd.read_file(r"output_folder\osm_infrastructure\osm_shops_and_amenities.shp")

#### For each destination for each type of graph do an accessibility analysis

In [14]:
for G in graphs.items():
    # Origins remain the same for each type of accessibility
    # insert origins
    origins = gpd.read_file(rinput_folder\cap_haitien_worldpop_pts2.shp")
    # snap the origins to the road graph
    snapped_origins = gn.pandana_snap(G[1], origins, source_crs = 'epsg:4326', target_crs = 'epsg:32619')
    # filter out origins with a distance more than 2km
    snapped_origins_filtered = snapped_origins[snapped_origins.NN_dist < 2000]
    originNodes = list(snapped_origins_filtered['NN'].unique())

    # snap the destinations to the road graph
    snapped_destinations = gn.pandana_snap(G[1], destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32619')
    destinationsNodes = list(snapped_destinations['NN'].unique())
    # Calculate OD Matrix
    OD_matrix = gn.calculate_OD(G[1], originNodes, destinationsNodes, fail_value=-1, weight='length')
    
    OD_df = pd.DataFrame(OD_matrix, columns = destinationsNodes , index = originNodes)
    
    # 60 minute threshold
    threshold_in_sec = 60 * 60
    
    # calculate accessibility
    accessibility_measure = OD_df[OD_df <= threshold_in_sec].count(axis=1)
    
    results = pd.DataFrame([originNodes, list(accessibility_measure)]).transpose()
    colName = "accessibility_measure"
    results.columns = ['NN', colName]
    
    output = snapped_origins_filtered.copy()
    output = pd.merge(output, results, on="NN")
    
    #convert travel_time_to_closest_facility to number
    output["accessibility_measure"] = pd.to_numeric(output["accessibility_measure"])
    
    # save a shapefile...
    destinations_gpd = gpd.GeoDataFrame(output, crs = "epsg:4326", geometry = 'geometry')
    destinations_gpd.to_file(fr"output_folder\cap_haitien_accessibility_economic_{G[0]}.shp")


SyntaxError: unexpected character after line continuation character (<ipython-input-14-b93bf47d6374>, line 4)